# Momentum Screener

###### This notebook is used to develop a custom momentum-factor equities screener which will be used to find a selection of top momentum stocks based on the following criteria:



1.) Volatility Adjusted Momentum: We want to select equities where momentum exists even after adjusting for volatility or beta. That way we aren't just going long beta, we want to be compensated with excess returns for going long equities with relatively higher volatility.

2.) Momentum Quality: The path to momentum is important. We want to favour equities with smooth price returns rather than jumpy. Small but consisent price returns are more favorable as they are more discrete and attract less attention and have been studied to outperform for longer periods and improve risk adjusted momentum factor returns.
 - We Use the FIP (Frog in the Pan) Algorithm which looks at % negative days/ % positive days as a measure of momentum quality
 
3.) Convexity: We also will be screening for equities which display price convexity in their time series. We can think of convexity as the acceleration of returns, in other words price is increasing at an increasing rate
 - Quadratic Regression will give us a positive coefficient if there is convexity in the price data
 
Ranking Method: For simplicity we will be placing equal weight across the three criteria of vol adjusted returns, momentum quality, and convexity. What this means is that equities will be ranked on an aggregate score of each their 3 measured features... So you just add the index number for all 3 into the net score. 

The End result will be a CSV file that gets uploaded to our local computer consisting of a list of tickers and their score going from best to worst.
 
 
Bonus Tip:

Diversification: Diversification is key with momentum. Diverisify across market caps, and use multiple methods at ranking your equities.


In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.linear_model import LinearRegression
import pandas_datareader as pdr

In [82]:
# Create a dataframe called 'sp500_tickers'
sp500_tickers = pd.read_csv('sa_tickers.csv')  # Assuming you have a file of S&P500 tickers
sp500_tickers.head()

,ticker
0,QUBT
1,RGTI
2,LX
3,CRDO
4,FOA


## Volatility Adjusted Returns

In [83]:
# Create a blank dataframe with the appropriate columns
df = pd.DataFrame(columns=['Ticker', 'Vol Adjusted Return'])

In [84]:
# Collect rows in a list for concatenation
rows_to_add = []

for ticker in sp500_tickers['ticker']:  # Iterate over the 'ticker' column in sp500_tickers dataframe
    try:
        # Retrieve 1 year of historical data
        data = yf.download(ticker, period='2y')  
        
        # Check if data is insufficient
        if data.empty or len(data) < 252:  
            raise ValueError(f"Insufficient data for ticker: {ticker}")

        # Calculate daily returns
        data['Daily Return'] = data['Close'].pct_change()

        # Calculate percent return over a 1-year period (252 trading days)
        data['Percent Return'] = ((data['Close'] - data['Close'].shift(252)) / data['Close'].shift(252)) * 100  

        # Rolling window for std dev (daily returns)
        window = 252  

        # Calculate rolling standard deviation of daily returns
        data['Rolling_Std'] = data['Daily Return'].rolling(window).std()

        # Annualize rolling standard deviation to get historical annual volatility
        data['Rolling_Hist_Vol'] = data['Rolling_Std'] * np.sqrt(252)

        # Calculate volatility-adjusted returns
        data['Vol_Adjusted_Return'] = ((data['Percent Return'])/100) / (data['Rolling_Hist_Vol'])

        # Retrieve the last value of Vol Adjusted Return
        vol_adjusted_return = data['Vol_Adjusted_Return'].iat[-1]

        # Add the result to the list
        rows_to_add.append({'Ticker': ticker, 'Vol Adjusted Return': vol_adjusted_return})

    except Exception as e:
        print(f"Error retrieving data for {ticker}: {str(e)}")

# Concatenate all rows into the DataFrame at once
df = pd.concat([pd.DataFrame(rows_to_add)], ignore_index=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- BRK.A: No data found, symbol may be delisted
Error retrieving data for BRK.A: Insufficient data for ticker: BRK.A
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%**********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

...

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Error retrieving data for HMENF: Insufficient data for ticker: HMENF
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [85]:
df = df.sort_values('Vol Adjusted Return', ascending=False)

In [86]:
# Create a csv file with the results
df.to_csv("Momentum Screen Results.csv", index=False) # set index = to false, or else it'll create a column with index

In [87]:
df1 = pd.read_csv("Momentum Screen Results.csv")

In [125]:
df1.head()

,Ticker,Vol Adjusted Return,Index2
0,WGS,14.859416,0
1,PSIX,14.249281,1
2,APP,9.262477,2
3,RCAT,7.986277,3
4,KINS,7.412391,4


## Assorted by Regular Momentum Returns

In [67]:
# Collect rows in a list for concatenation
rows_to_add_5 = []

for ticker in sp500_tickers['ticker']:  # Iterate over the 'ticker' column in sp500_ticker dataframe
    try:
        data = yf.download(ticker, period='2y')  
        if data.empty or len(data) < 250:  # Check if data is insufficient
            raise ValueError(f"Insufficient data for ticker: {ticker}")

        data['Percent Return'] = ((data['Close'] - data['Close'].shift(252)) / data['Close'].shift(252)) * 100
        percent_return = data['Percent Return'].iloc[-1]  # Get return for the last trading day
         
        # Add the result to the list
        rows_to_add_5.append({'Ticker': ticker, 'Percent Return': percent_return})
    
    except Exception as e:
        print(f"Error retrieving data for {ticker}: {str(e)}")

# Concatenate all rows into the DataFrame at once
df = pd.concat([pd.DataFrame(rows_to_add_5)], ignore_index=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [68]:
df = df.sort_values('Percent Return', ascending=False)

In [69]:
df.head()

,Ticker,Percent Return
6,NVDA,145.678203
24,AVGO,115.136779
5,TSLA,97.865922
11,WMT,71.242326
7,META,66.346344


In [7]:
# Create a csv file with the results
df.to_csv("Momentum Screen Results.csv", index=False) # set index = to false, or else it'll create a column with index

## Convexity of Returns

In [54]:
#only keep first 100 rows of the Momentum Screen Results 
df1 = df1.iloc[:100]

In [66]:
# Collect rows in a list for concatenation
rows_to_add_4 = []

# Initialize a results DataFrame
results_df = pd.DataFrame(columns=['Ticker', 'Convexity Ratio'])

for Ticker in df1['Ticker']:  # Iterate over the 'Ticker' column
    try:
        # Download 2 years of historical data
        data = yf.download(Ticker, period='2y')
        
        if data.empty or len(data) < 252:  # Check if data is insufficient
            raise ValueError(f"Insufficient data for ticker: {Ticker}")
        
        # Create a DataFrame for returns
        df = pd.DataFrame()
        df['Return'] = data['Adj Close'].pct_change()

        # Calculate the slope using a rolling window
        window = 50  # Adjust window size as needed
        df['Slope'] = df['Return'].rolling(window).apply(
            lambda x: np.polyfit(np.arange(len(x)), x, 1)[0], raw=True
        )

        # Compute the second derivative of the slope
        df['Second_Derivative'] = df['Slope'].diff()

        # Calculate the convexity ratio
        df['Convexity_Ratio'] = df['Second_Derivative'] / df['Slope']
        
        # Extract the most recent convexity ratio
        convexity_ratio = df['Convexity_Ratio'].iloc[-1]
        
        # Add the result to the list
        rows_to_add_4.append({'Ticker': Ticker, 'Convexity Ratio': convexity_ratio})
    
    except Exception as e:
        print(f"Error retrieving data for {Ticker}: {str(e)}")
        
# Concatenate all rows into the DataFrame at once
results_df = pd.concat([pd.DataFrame(rows_to_add_4)], ignore_index=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [56]:
df2 = results_df.sort_values(by='Convexity Ratio', ascending=False)

In [79]:
df2.head()

,Ticker,Convexity Ratio
46,MRK,7.440757
48,INTC,3.696518
44,PEP,2.642901
24,TXN,1.623321
20,XOM,1.375793


In [83]:
#Create a new csv file with our FIP results
df2.to_csv('Convexity_Results.csv', index=False)






### 

## Convexity Based on Quadratic Regression

In [102]:
df1 = df1.iloc[:200]

In [103]:
rows_to_add_2 = []

# Initialize an empty DataFrame to store results
df_convexity = pd.DataFrame(columns=['Ticker', 'Convexity Score'])

for ticker in df1['Ticker']:
    try:
        # Download historical data
        data = yf.download(ticker, period='2y')
        
        # Ensure there is sufficient data
        if data.empty or len(data) < 252:  
            raise ValueError(f"Insufficient data for ticker: {ticker}")
        
        # Calculate daily returns (percentage change)
        data['Return'] = data['Close'].pct_change() * 100

        # Drop the first row since the return for the first day is NaN
        data = data.dropna(subset=['Return'])
        
        # Create an ordinal time variable (index of the data)
        data.loc[:, 'Time'] = np.arange(len(data))
        
        # Prepare the independent variables (Time and Time^2 for quadratic regression)
        X = np.vstack([data['Time'], data['Time']**2]).T
        y = data['Return'].values
        
        # Perform a quadratic regression
        model = LinearRegression().fit(X, y)
        
        # The convexity is the coefficient of the squared time term
        convexity = model.coef_[1]  # This is the coefficient of Time^2 (quadratic term)
        
        # Add the result to the list
        rows_to_add_2.append({'Ticker': ticker, 'Convexity Score': convexity})
    
    except Exception as e:
        print(f"Error retrieving data for {ticker}: {str(e)}")
        
# Concatenate all rows into the DataFrame at once
df_convexity = pd.concat([pd.DataFrame(rows_to_add_2)], ignore_index=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


C:\Users\eabiz\AppData\Local\Temp\ipykernel_29936\3755661378.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'Time'] = np.arange(len(data))
C:\Users\eabiz\AppData\Local\Temp\ipykernel_29936\3755661378.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'Time'] = np.arange(len(data))


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


C:\Users\eabiz\AppData\Local\Temp\ipykernel_29936\3755661378.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'Time'] = np.arange(len(data))
C:\Users\eabiz\AppData\Local\Temp\ipykernel_29936\3755661378.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'Time'] = np.arange(len(data))


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...

C:\Users\eabiz\AppData\Local\Temp\ipykernel_29936\3755661378.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'Time'] = np.arange(len(data))
C:\Users\eabiz\AppData\Local\Temp\ipykernel_29936\3755661378.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'Time'] = np.arange(len(data))


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


C:\Users\eabiz\AppData\Local\Temp\ipykernel_29936\3755661378.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'Time'] = np.arange(len(data))
C:\Users\eabiz\AppData\Local\Temp\ipykernel_29936\3755661378.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'Time'] = np.arange(len(data))


[*********************100%***********************]  1 of 1 completed


C:\Users\eabiz\AppData\Local\Temp\ipykernel_29936\3755661378.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'Time'] = np.arange(len(data))


In [104]:
df_convexity = df_convexity.sort_values('Convexity Score', ascending=False)

In [105]:
df_convexity.reset_index(drop=True, inplace=True)

In [106]:
df_convexity.head()

,Ticker,Convexity Score
0,LUNR,0.000061
1,QUBT,0.000061
2,RGTI,0.000052
3,KULR,0.000034
4,EOSE,0.000034


In [107]:
df_convexity.to_csv("Convexity_Score_Quadratic_Method.csv")




####

### Momentum Quality - FIP (Frog in the Pan Score)

ID or FIP Score
ID = sign x [% Negative - % Positive]

In [108]:
#only keep first 100 rows of the Momentum Screen Results
df1 = df1.iloc[:200]

In [109]:
# Create a blank dataframe with the appropriate columns
df_FIP = pd.DataFrame(columns=['Ticker', 'FIP_Score'])

In [110]:
rows_to_add_3 = []

for Ticker in df1['Ticker']:  # Iterate over the 'ticker' column in sp500_ticker df
    try:
        data = yf.download(Ticker, period='2y')  # Retrieve data. Ensure > 252 days else there'll be an error
        if data.empty or len(data) < 252:  # Check if data is insufficient
            raise ValueError(f"Insufficient data for ticker: {ticker}")
        
        data['Percent Return'] = (data['Close'].pct_change())*100
        if data['Percent Return'].cumsum().iat[-1] > 0:
            sign = 1
        elif data['Return'].cumsum().iat[-1] < 0:
            sign = -1
        else:
            sign = 0
        positive_days = (data['Percent Return'] > 0).sum()
        negative_days = (data['Percent Return'] < 0).sum()
        flat_days = (data['Percent Return'] == 0).sum()
        pct_positive = (round((positive_days/len(data)), 4))
        pct_negative = (round((negative_days/len(data)), 4))
        FIP_Score = round(sign*(pct_negative - pct_positive),4)
                
        # Add the result to the list
        rows_to_add_3.append({'Ticker': Ticker, 'FIP Score': FIP_Score})
        
    except Exception as e:
        print(f"Error retrieving data for {ticker}: {str(e)}")
        
df_FIP = pd.concat([pd.DataFrame(rows_to_add_3)], ignore_index=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [111]:
df_FIP = df_FIP.sort_values('FIP Score', ascending=True) # ascending = True since the best scores are negative

In [112]:
df_FIP.reset_index(drop=True, inplace=True)

In [113]:
df_FIP.head()

,Ticker,FIP Score
0,GDDY,-0.1869
1,HWM,-0.1650
2,EME,-0.1590
3,VCTR,-0.1571
4,UNM,-0.1511


In [114]:
# Create a new csv file with our FIP results
df_FIP.to_csv('Momentum_FIP_Results.csv', index=False)






### 

# Combine Dataframes and Rank Tickers

In [115]:
#Create column of index #
df1['Index2'] = df1.index

#Create column of index #
df_FIP['Index1'] = df_FIP.index

# create column of index # 
df_convexity['Index3'] = df_convexity.index

In [116]:
# Perform an inner join to include only matching tickers
merged = pd.merge(df_FIP, df1, on='Ticker', how='inner')

### Next cell is only necessary if adding a third filter to the ranking:

In [117]:
# Merge the result with the third DataFrame
merged = pd.merge(merged, df_convexity, on='Ticker', how='inner')

In [121]:
merged.head()

,Ticker,FIP Score,Index1,Vol Adjusted Return,Index2,Convexity Score,Index3,Score
0,GDDY,-0.1869,0,3.780848,79,-4.787073e-06,166,245
1,HWM,-0.1650,1,3.887278,75,-2.680006e-07,116,192
2,EME,-0.1590,2,4.068671,65,-2.155669e-06,143,210
3,VCTR,-0.1571,3,2.892325,142,-1.057983e-06,129,274
4,UNM,-0.1511,4,3.184801,109,3.552185e-06,58,171


In [126]:
# Calculate the score (sum of indices)
merged['Score'] = merged['Index1'] + merged['Index2'] + merged['Index3']
# Sort the result by the score
sorted_df = merged.sort_values('Score', ascending=True)
# Select relevant columns for the output
result = sorted_df[['Ticker', 'Score']]

In [127]:
result.head()

,Ticker,Score
36,APP,53
54,LFVN,109
40,AGX,114
9,TRGP,116
58,HOOD,125


In [130]:
result.to_csv("Momentum_Ranked.csv",index=False)






#####